In [1]:
import nltk
import random
from collections import Counter

# Завантаження корпусу (приклад для Brown)
nltk.download('brown')
from nltk.corpus import brown

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


In [2]:
#nltk.corpus.conll2000
nltk.download('conll2000')

from nltk.corpus import conll2000

[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Unzipping corpora/conll2000.zip.


In [3]:
tagged_sents = list(conll2000.tagged_sents())

In [4]:
# Перемішування та розділення на навчальну і тестову вибірки
random.seed(42)
random.shuffle(tagged_sents)

split_point = int(len(tagged_sents) * 0.8)
train_sents = tagged_sents[:split_point]
test_sents = tagged_sents[split_point:]

# Запис у файли
def write_tagged_sentences(sentences, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        for sentence in sentences:
            for word, tag in sentence:
                f.write(f"{word}\t{tag}\n")
            f.write("\n")  # Порожній рядок між реченнями

write_tagged_sentences(train_sents, "conll2000_training.pos")
write_tagged_sentences(test_sents, "conll2000_test.pos")

# Створення словника - виправлена версія
word_counter = Counter()
for sentence in train_sents:
    for word, _ in sentence:
        word_counter[word] += 1

# Збереження слів, які з'являються принаймні двічі
vocab = {word for word, count in word_counter.items() if count >= 2}

with open("conll2000_vocab.txt", 'w', encoding='utf-8') as f:
    for word in sorted(vocab):
        f.write(f"{word}\n")

# Створення файлу тестових слів
with open("conll2000_test_words.txt", 'w', encoding='utf-8') as f:
    for sentence in test_sents:
        for word, _ in sentence:
            f.write(f"{word}\n")
        f.write("\n")  # Порожній рядок між реченнями

In [5]:
import string

# Символи пунктуації
punct = set(string.punctuation)

# Морфологічні правила, що використовуються для класифікації невідомих слів
noun_suffix = ["action", "age", "ance", "cy", "dom", "ee", "ence", "er", "hood", "ion", "ism", "ist", "ity", "ling", "ment", "ness", "or", "ry", "scape", "ship", "ty"]
verb_suffix = ["ate", "ify", "ise", "ize"]
adj_suffix = ["able", "ese", "ful", "i", "ian", "ible", "ic", "ish", "ive", "less", "ly", "ous"]
adv_suffix = ["ward", "wards", "wise"]


In [6]:
def assign_unk(tok):
    """
    Призначення міток для невідомих слів
    """
    # Цифри
    if any(char.isdigit() for char in tok):
        return "--unk_digit--"

    # Пунктуація
    elif any(char in punct for char in tok):
        return "--unk_punct--"

    # Великі літери
    elif any(char.isupper() for char in tok):
        return "--unk_upper--"

    # Іменники
    elif any(tok.endswith(suffix) for suffix in noun_suffix):
        return "--unk_noun--"

    # Дієслова
    elif any(tok.endswith(suffix) for suffix in verb_suffix):
        return "--unk_verb--"

    # Прикметники
    elif any(tok.endswith(suffix) for suffix in adj_suffix):
        return "--unk_adj--"

    # Прислівники
    elif any(tok.endswith(suffix) for suffix in adv_suffix):
        return "--unk_adv--"

    return "--unk--"

In [7]:
def preprocess(vocab, data_fp):
    """
    Попередня обробка даних
    """
    orig = []
    prep = []

    # Додаю спеціальний токен для кінця речення
    if "--n--" not in vocab:
        vocab["--n--"] = len(vocab)

    # Читання даних
    with open(data_fp, "r") as data_file:
        for cnt, word in enumerate(data_file):
            # Кінець речення
            if not word.split():
                orig.append(word.strip())
                word = "--n--"
                prep.append(word)
                continue

            # Обробка невідомих слів
            elif word.strip() not in vocab:
                orig.append(word.strip())
                word = assign_unk(word.strip())  # Виправлення: додано strip()
                prep.append(word)
                continue

            else:
                orig.append(word.strip())
                prep.append(word.strip())

    assert(len(orig) == len(open(data_fp, "r").readlines()))
    assert(len(prep) == len(open(data_fp, "r").readlines()))

    return orig, prep


In [8]:
import numpy as np
import pandas as pd
from collections import defaultdict
import math

# Завантаження навчального корпусу
with open("/content/conll2000_training.pos", 'r') as f:
    training_corpus = f.readlines()

# Завантаження словника
with open("/content/conll2000_vocab.txt", 'r') as f:
    voc_l = f.read().split('\n')

# Створення словника з індексами для слів
vocab = {}
for i, word in enumerate(sorted(voc_l)):
    vocab[word] = i

# Завантаження тестового корпусу
with open("/content/conll2000_test.pos", 'r') as f:
    y = f.readlines()

# Попередня обробка тестових слів
_, prep = preprocess(vocab, "/content/conll2000_test_words.txt")


In [9]:
training_corpus[0]

'No\tDT\n'

In [10]:
y[0]

'But\tCC\n'

In [11]:
# Виправлення 2: Виправлена функція get_word_tag
def get_word_tag(line, vocab):
    """
    Отримання слова та його тегу з рядка корпусу.
    """
    if not line.split():
        word = "--n--"
        tag = "--s--"
        return word, tag  # Виправлення: додано повернення значення
    else:
        parts = line.split()
        if len(parts) >= 2:
            word, tag = parts[0], parts[1]
            if word not in vocab:
                # Обробка невідомих слів
                word = assign_unk(word)
            return word, tag
        else:
            # Обробка некоректних рядків
            return "--n--", "--s--"  # Виправлення: додано повернення для некоректних рядків


def create_dictionaries(training_corpus, vocab):
    """
    Створення словників частот.
    """
    emission_counts = defaultdict(int)
    transition_counts = defaultdict(int)
    tag_counts = defaultdict(int)

    # Початковий тег
    prev_tag = '--s--'

    for word_tag in training_corpus:
        word, tag = get_word_tag(word_tag, vocab)

        # Збільшення лічильника переходів
        transition_counts[(prev_tag, tag)] += 1

        # Збільшення лічильника емісій
        emission_counts[(tag, word)] += 1

        # Збільшення лічильника тегів
        tag_counts[tag] += 1

        # Оновлення попереднього тегу
        prev_tag = tag

    return emission_counts, transition_counts, tag_counts


In [12]:
def create_transition_matrix(alpha, tag_counts, transition_counts):
    """
    Створення матриці переходів A.
    """
    all_tags = sorted(tag_counts.keys())
    num_tags = len(all_tags)

    A = np.zeros((num_tags, num_tags))

    for i in range(num_tags):
        for j in range(num_tags):
            key = (all_tags[i], all_tags[j])

            count = 0
            if key in transition_counts:
                count = transition_counts[key]

            count_prev_tag = tag_counts[all_tags[i]]

            # Застосування згладжування
            A[i, j] = (count + alpha) / (count_prev_tag + alpha * num_tags)

    return A

In [13]:
def create_emission_matrix(alpha, tag_counts, emission_counts, vocab):
    """
    Створення матриці емісій B.
    """
    all_tags = sorted(tag_counts.keys())
    num_tags = len(tag_counts)
    num_words = len(vocab)

    B = np.zeros((num_tags, num_words))

    for i in range(num_tags):
        for j in range(num_words):
            key = (all_tags[i], vocab[j])

            count = 0
            if key in emission_counts:
                count = emission_counts[key]

            count_tag = tag_counts[all_tags[i]]

            # Застосування згладжування
            B[i, j] = (count + alpha) / (count_tag + alpha * num_words)

    return B


In [14]:
# Виправлення 3: Оновлення функцій Вітербі для роботи з невідомими словами
def initialize(states, tag_counts, A, B, corpus, vocab):
    """
    Ініціалізація алгоритму Вітербі.
    """
    num_tags = len(tag_counts)

    # Ініціалізація матриць best_probs та best_paths
    best_probs = np.zeros((num_tags, len(corpus)))
    best_paths = np.zeros((num_tags, len(corpus)), dtype=int)

    # Індекс початкового стану
    s_idx = states.index("--s--")

    # Заповнення першого стовпця best_probs
    word = corpus[0]
    word_idx = vocab.get(word, vocab.get("--unk--", 0))  # Виправлення: обробка невідомих слів

    for i in range(num_tags):
        best_probs[i, 0] = math.log(A[s_idx, i]) + math.log(B[i, word_idx])

    return best_probs, best_paths

def viterbi_forward(A, B, corpus, best_probs, best_paths, vocab):
    """
    Пряме проходження алгоритму Вітербі.
    """
    num_tags = best_probs.shape[0]

    for i in range(1, len(corpus)):
        word = corpus[i]
        word_idx = vocab.get(word, vocab.get("--unk--", 0))  # Виправлення: обробка невідомих слів

        for j in range(num_tags):
            best_prob_i = float("-inf")
            best_path_i = None

            for k in range(num_tags):
                prob = best_probs[k, i-1] + math.log(A[k, j]) + math.log(B[j, word_idx])

                if prob > best_prob_i:
                    best_prob_i = prob
                    best_path_i = k

            best_probs[j, i] = best_prob_i
            best_paths[j, i] = best_path_i

    return best_probs, best_paths


def viterbi_backward(best_probs, best_paths, corpus, states):
    """
    Зворотне проходження алгоритму Вітербі.
    """
    m = best_paths.shape[1]
    z = [None] * m
    pred = [None] * m

    # Знаходження найкращого тегу для останнього слова
    best_prob_for_last_word = float('-inf')

    for k in range(best_probs.shape[0]):
        if best_probs[k, m-1] > best_prob_for_last_word:
            best_prob_for_last_word = best_probs[k, m-1]
            z[m-1] = k

    pred[m-1] = states[z[m-1]]

    # Зворотне проходження для знаходження найкращих тегів
    for i in range(m-2, -1, -1):
        z[i] = best_paths[z[i+1], i+1]
        pred[i] = states[z[i]]

    return pred


In [15]:
def compute_accuracy(pred, y):
    """
    Обчислення точності моделі POS-тегування.
    """
    num_correct = 0
    total = 0

    for prediction, y_item in zip(pred, y):
        y_item = y_item.strip()
        word_tag_tuple = y_item.split('\t')

        if len(word_tag_tuple) != 2:
            continue

        word, tag = word_tag_tuple

        if tag == prediction:
            num_correct += 1

        total += 1

    return num_correct / total


In [16]:
# Виправлення 4: Код для запуску моделі
# Створення словників
emission_counts, transition_counts, tag_counts = create_dictionaries(training_corpus, vocab)

# Отримання списку всіх тегів
states = sorted(tag_counts.keys())


In [17]:
# Параметр згладжування
alpha = 0.001

# Створення індексованого словника слів
word_to_index = {}
for i, word in enumerate(sorted(vocab.keys())):
    word_to_index[word] = i

# Додавання спеціальних токенів
for special_token in ["--n--", "--unk--", "--unk_digit--", "--unk_punct--", "--unk_upper--",
                      "--unk_noun--", "--unk_verb--", "--unk_adj--", "--unk_adv--"]:
    if special_token not in word_to_index:
        word_to_index[special_token] = len(word_to_index)


In [18]:
# Створення матриць переходів та емісій
A = create_transition_matrix(alpha, tag_counts, transition_counts)
B = create_emission_matrix(alpha, tag_counts, emission_counts, list(word_to_index.keys()))

In [19]:
# Ініціалізація алгоритму Вітербі
best_probs, best_paths = initialize(states, tag_counts, A, B, prep, word_to_index)


In [20]:
# Пряме проходження
best_probs, best_paths = viterbi_forward(A, B, prep, best_probs, best_paths, word_to_index)


In [21]:
# Зворотне проходження для отримання найкращої послідовності тегів
pred = viterbi_backward(best_probs, best_paths, prep, states)

# Обчислення точності
accuracy = compute_accuracy(pred, y)
print(f"Точність моделі: {accuracy:.4f}")


Точність моделі: 0.9580


In [22]:
import nltk
from nltk import word_tokenize

# Завантаження необхідних ресурсів
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('conll2000')

# Приклад тексту
text = "The market's pulse will be checked through multiple diagnostic tools this quarter."

# Токенізація тексту
tokens = word_tokenize(text)

# POS-тегування з використанням NLTK
nltk_tags = nltk.pos_tag(tokens)

print("Результат POS-тегування NLTK:")
for word, tag in nltk_tags:
    print(f"{word}: {tag}")

# Оцінка точності NLTK POS-tagger на корпусі treebank
train_data = conll2000.tagged_sents()[:3000]
test_data = conll2000.tagged_sents()[3000:]

# Навчання NLTK тегера
from nltk.tag import UnigramTagger, BigramTagger
unigram_tagger = UnigramTagger(train_data)
bigram_tagger = BigramTagger(train_data, backoff=unigram_tagger)

# Оцінка точності
nltk_accuracy = bigram_tagger.evaluate(test_data)
print(f"Точність NLTK BigramTagger: {nltk_accuracy:.4f}")

# Порівняння з нашою реалізацією
print(f"Точність нашої реалізації: {accuracy:.4f}")
print(f"Різниця: {abs(accuracy - nltk_accuracy):.4f}")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!


Результат POS-тегування NLTK:
The: DT
market: NN
's: POS
pulse: NN
will: MD
be: VB
checked: VBN
through: IN
multiple: JJ
diagnostic: JJ
tools: NNS
this: DT
quarter: NN
.: .


<ipython-input-22-96f66dc204d6>:34: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  nltk_accuracy = bigram_tagger.evaluate(test_data)


Точність NLTK BigramTagger: 0.8590
Точність нашої реалізації: 0.9580
Різниця: 0.0989
